In [ ]:
!wget https://github.com/t-davidson/hate-speech-and-offensive-language/raw/master/data/labeled_data.p

!unzip ../input/jigsaw-toxic-comment-classification-challenge/train.csv.zip
!unzip ../input/jigsaw-toxic-comment-classification-challenge/test.csv.zip
!unzip ../input/jigsaw-toxic-comment-classification-challenge/test_labels.csv.zip

!pip install pandarallel

In [ ]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
from pandarallel import pandarallel

In [ ]:
pandarallel.initialize(progress_bar=True)

In [ ]:
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

train_df = pd.read_csv(os.path.join('./', 'train.csv'))
test_df = pd.read_csv(os.path.join('./', 'test.csv')).merge(pd.read_csv(os.path.join('./', 'test_labels.csv')), on='id')
test_df.drop(test_df[test_df[label_cols].min(axis=1) == -1].index, inplace=True)

In [ ]:
df = pd.concat([train_df, test_df], ignore_index=True)
df['toxic?'] = df.parallel_apply(lambda row: row[label_cols].any(), axis=1)

In [ ]:
df.drop(label_cols, inplace=True, axis=1)
df.drop('id', inplace=True, axis=1)
df.rename(columns={'comment_text': 'text'}, inplace=True)

In [ ]:
df

In [ ]:
df2 = pd.read_pickle('labeled_data.p')
df2.rename(columns={'tweet': 'text'}, inplace=True)
df2['toxic?'] = df2.parallel_apply(lambda row: int(row['class'] != 2), axis=1)
df2 = df2[['text', 'toxic?']]

In [ ]:
df2

In [ ]:
df = pd.concat([df, df2], ignore_index=True)

In [ ]:
df.to_pickle('df.pkl')